In [ ]:
# Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("elyza/Llama-3-ELYZA-JP-8B")
# model = AutoModelForCausalLM.from_pretrained("elyza/Llama-3-ELYZA-JP-8B")

# Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B")
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-3B")


# # Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct", trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3.5-mini-instruct", trust_remote_code=True)

# Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("Qwen/QwQ-32B-Preview")
# model = AutoModelForCausalLM.from_pretrained("Qwen/QwQ-32B-Preview")

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 17/17 [00:08<00:00,  1.96it/s]


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig

model_name = "elyza/Llama-3-ELYZA-JP-8B"
# model_name = "Qwen/Qwen2.5-3B"
# model_name = "microsoft/Phi-3.5-mini-instruct"
# model_name = "Qwen/QwQ-32B-Preview"
tokenizer = AutoTokenizer.from_pretrained(model_name)

quant_config = BitsAndBytesConfig(
    # load_in_8bit=True  # 8ビット量子化を指定,
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    
    bnb_4bit_compute_dtype=torch.bfloat16 #bfloatはfloatに比べて、精度よりも数値が表現できる桁数の幅（ダイナミックレンジ）を重視した設計...らしい
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config, #量子化を行う
    torch_dtype="auto",
    device_map="auto",
)
model.eval()

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:12<00:00,  3.02s/it]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Ll

In [3]:


DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人のアシスタントです。特に指示が無い場合は、常に日本語で回答してください。"
text = "仕事の熱意を取り戻すためのアイデアを5つ挙げてください。"
assistant_output = "そんなもんねーよハゲ！"


messages = [
    {"role": "system", "content": DEFAULT_SYSTEM_PROMPT},
    {"role": "user", "content": text},
    {"role": "assistant", "content": assistant_output},
]
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
token_ids = tokenizer.encode(
    prompt, add_special_tokens=False, return_tensors="pt"
)

print(prompt)
print(token_ids)
# with torch.no_grad():
#     output_ids = model.generate(
#         token_ids.to(model.device),
#         max_new_tokens=1200,
#         do_sample=True,
#         temperature=0.6,
#         top_p=0.9,
#     )
# output = tokenizer.decode(
#     output_ids.tolist()[0][token_ids.size(1):], skip_special_tokens=True
# )
# print(output)


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

あなたは誠実で優秀な日本人のアシスタントです。特に指示が無い場合は、常に日本語で回答してください。<|eot_id|><|start_header_id|>user<|end_header_id|>

仕事の熱意を取り戻すためのアイデアを5つ挙げてください。<|eot_id|><|start_header_id|>assistant<|end_header_id|>

そんなもんねーよハゲ！<|eot_id|><|start_header_id|>assistant<|end_header_id|>


tensor([[128000, 128006,   9125, 128007,    271,  30591, 112568,  15682, 124097,
         103350,  16556, 104622, 106241,  26854, 102433, 107707,  39880,  57207,
         105335,  52414,  38641,   1811,  66378,  20230,  64467,  20379,  29295,
          43568,  16995, 126513,   5486,  40053,  20230, 102433, 102158,  16556,
         113925,  39926,  72315,   1811, 128009, 128006,    882, 128007,    271,
         117876,  16144, 107969,  37689,  30512, 108167, 114941,  17663, 124122,
         111090,  68408,  39880,  30512,     20,  59739, 123680, 102639,  38144,
          72315,   1811, 128009, 128006,  78191, 128007,    271, 107943,  32977,
          25827, 101832,  38248,    

In [1]:
#学習済みモデルを利用してみる
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1" #これによりgpu1がcuda:0として扱われるようになるらしい、複数台利用したいときは0,1などにすればよい
from generate_graph_local import Graph_Generator
# graph_generator = Graph_Generator(USE_LOCAL_LLM=True,model_name="Qwen/Qwen2.5-3B",tokenizer_name ="Qwen/Qwen2.5-3B" )
graph_generator = Graph_Generator(USE_LOCAL_LLM=True,model_name="Phi-3.5-mini-instruct-lora_ft",tokenizer_name ="microsoft/Phi-3.5-mini-instruct" )
# graph_generator = Graph_Generator(USE_LOCAL_LLM=True,model_name="elyza/Llama-3-ELYZA-JP-8B",tokenizer_name ="elyza/Llama-3-ELYZA-JP-8B" )

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.25it/s]


In [2]:
from bs4 import  BeautifulSoup
import os


saved_html_list = os.listdir("./data/recipe_htmls/")
len(saved_html_list)
for html in saved_html_list[10:11]:
    html = f"./data/recipe_htmls/{html}"
    with open(html,"r",encoding="utf-8") as f:
        graph = graph_generator.generate_graph(url=None,html=f)
        print(graph)

text_content タイトル: 美味しくてびっくり！ローストビーフ♪赤ワインソース レシピ・作り方

材料:
ローストビーフ用かたまり肉 400ｇ～500ｇ
塩 小さじ１/２
粗引き黒胡椒 少々
ローリエ 2枚
ニンニク　スライス １かけ分
オリーブオイル 大さじ１
★赤ワインソース★ 
赤ワイン 300ｃｃ
玉ねぎ　くし切り １/２個
蜂蜜 大さじ２
醤油 大さじ２
コンソメキューブ 1ケ
★バター 10ｇ

手順:
牛肉に塩、黒コショウをすりこみ、ローリエ、にんにく、オリーブオイル、と一緒にビニール袋に入れ１時間ほど寝かせます。
テフロン加工のフライパンで、肉の表面をに焦げ目を付けるように焼きます。(ローリエ、ニンニクも一緒に、)強火で、一面30秒づつ焼きつけます。※トングを使うと簡単です。
鍋に赤ワイン、玉ねぎ入れを煮立たせます。アルコールが飛んだら、コンソメ、醤油、蜂蜜を加え溶かします。
2の焼いた肉とニンニク、ローリエを加え、クッキングシートのお落しブタと鍋の蓋をして、中火～中弱火で4分加熱します。火を消し蓋をしたまま20分おきます。(予熱で火が通ります)
肉を取り出し、アルミホイルに包み冷めるまで待ちます。(熱いうちに切り分けるとドリップが出ます。)
(ソースの仕上げ)ローリエ、玉ねぎを取り除き、あくを取りながら２/３くらいまで煮詰めます。仕上げにバターを加えます。器に入れローストビーフに添えます。


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


output_str:
 Knowledge Graph:

Node 1: ローストビーフ用かたまり肉
Type: ingredient
Quantity: 400～500g

Node 2: 塩
Type: ingredient
Quantity: 小さじ1/2

Node 3: 粗引き黒胡椒
Type: ingredient
Quantity: 少々

Node 4: ローリエ
Type: ingredient
Quantity: 2枚

Node 5: ニンニク
Type: ingredient
Quantity: １かけ分

Node 6: オリーブオイル
Type: ingredient
Quantity: 大さじ1

Node 7: 赤ワイン
Type: ingredient
Quantity: 300cc

Node 8: 玉ねぎ
Type: ingredient
Quantity: 1/2個

Node 9: 蜂蜜
Type: ingredient
Quantity: 大さじ2

Node 10: 醤油
Type: ingredient
Quantity: 大さじ2

Node 11: コンソメキューブ
Type: ingredient
Quantity: 1個

Node 12: バター
Type: ingredient
Quantity: 10g

Node 13: ローストビーフ
Type: intermediate

Node 14: 赤ワインソース
Type: intermediate

Edge 1: ローストビーフ用かたまり肉 - ローストビーフ（焼く）
Edge 2: 塩 - ローストビーフ（下味付ける）
Edge 3: 粗引き黒胡椒 - ローストビーフ（下味付ける）
Edge 4: ローリエ - ローストビーフ（ローストする）
Edge 5: ニンニク - 赤ワインソース（煮込む）
Edge 6: オリーブオイル - ローストビーフ（焼く）
Edge 7: 赤ワイン - 赤ワインソース（溶かす）
Edge 8: 玉ねぎ - 赤ワインソース（煮込む）
Edge 9: 蜂蜜 - 赤ワインソース（煮込む）
Edge 10: 醤油 - 赤ワインソース（煮込む）
Edge 11: コンソメキューブ - 赤ワインソース（煮込む）
Edge 1

In [ ]:
#GPT4oの良質なoutputを利用する
def parse_to_json(input_string):
    # TODO: 色々なパターンのテキストに対応する
    # ノードのセクションを解析
    node_pattern = re.compile(r"Node (\d+): ([^\n]+)\nType: ([^\n]+)(?:\nQuantity: ([^\n]+))?")
    nodes = []
    for match in node_pattern.finditer(input_string):
        groups = match.groups() # example: ('1', '鶏もも肉', 'ingredient', '1枚(200g)')
        
        # quantityがある場合
        if len(groups) == 4:
            _, node_id, node_type, quantity = groups
            node = {"id": node_id, "type": node_type, "quantity": quantity}
        
        # quantityがない場合
        else:
            _, node_id, node_type = groups
            node = {"id": node_id, "type": node_type}
        
        nodes.append(node)
    
    # エッジのセクションを解析
    edge_pattern = re.compile(r"Edge \d+: ([^\s]+) - ([^\s]+) ?[\(（]([^)）]+)[\)）]")
    edges = []
    for match in edge_pattern.finditer(input_string):
        source, target, action = match.groups()
        edges.append({"source": source, "target": target, "action": action})

    # JSONオブジェクトを作成
    result = {
        "nodes": nodes,
        "edges": edges
    }

    return result

import os
import json
import re

created_recipe_jsons = []
GPT_outputs = os.listdir("./data/GPT_messages_outputs")
for _json in GPT_outputs:
    with open(f"./data/GPT_messages_outputs/{_json}","r",encoding="utf-8") as f:
        _json = json.load(f)
        _json = parse_to_json(_json["output_str"])
        created_recipe_jsons.append(_json)


with open(f"./data/recipe_graph_jsons/GPT4o.json","w",encoding="utf-8") as f:
    json.dump(created_recipe_jsons,f)

FileNotFoundError: [Errno 2] No such file or directory: './data/recipe_graph_jsons/GPT4o.json'